In [160]:
import tensorflow as tf
import numpy as np
import pandas as pd
import glob

In [161]:
def present_tensorboard_logs(stage='feature_selection'):
    # Create dict of values
    df_dict = {
        'run_name': [],
        'max_binary_accuracy': [],
        'max_binary_accuracy_epoch': [],
        'max_precision': [],
        'max_precision_epoch': [],
        'max_recall': [],
        'max_recall_epoch': [],
    }

    if stage == 'feature_selection':
    # Collect the validation filepaths
        filepaths = glob.glob('./tensorboard/feature_selection/*/validation/*')
    elif stage == 'model_selection':
        filepaths = glob.glob('./tensorboard/model_testing/*/validation/*')
    elif stage == 'model_tuning':
        filepaths = glob.glob('./tensorboard/model_tuning/*/*/*/validation/*')
        print(filepaths)

    # Iterate over the filepaths
    for filepath in filepaths:
        if stage == 'model_tuning':
            string_arr = run_name = filepath.split('/')
            run_name = f'{string_arr[3]}_{string_arr[4]}'
        else:
            run_name = filepath.split('/')[3]
        # Set the empty arrays
        epoch_binary_accuracy_arr = []
        epoch_precision_arr = []
        epoch_recall_arr = []

        # Initialise tf iterator to iterate over the log file
        for event in tf.compat.v1.train.summary_iterator(filepath):
            for value in event.summary.value:
                if value.tag == 'epoch_binary_accuracy':
                    epoch_binary_accuracy_arr.append(tf.make_ndarray(value.tensor))
                elif value.tag == 'epoch_precision':
                    epoch_precision_arr.append(tf.make_ndarray(value.tensor))
                elif value.tag == 'epoch_recall':
                    epoch_recall_arr.append(tf.make_ndarray(value.tensor))

        # Compute max and epoch max
        epoch_binary_accuracy_max = np.max(epoch_binary_accuracy_arr)
        epoch_binary_accuracy_max_epoch = epoch_binary_accuracy_arr.index(epoch_binary_accuracy_max)
        epoch_precision_max = np.max(epoch_precision_arr)
        epoch_precision_max_epoch = epoch_precision_arr.index(epoch_precision_max)
        epoch_recall_max = np.max(epoch_recall_arr)
        epoch_recall_max_epoch = epoch_recall_arr.index(epoch_recall_max)

        # Update dict
        df_dict['run_name'].append(run_name)
        df_dict['max_binary_accuracy'].append(epoch_binary_accuracy_max)
        df_dict['max_binary_accuracy_epoch'].append(epoch_binary_accuracy_max_epoch)
        df_dict['max_precision'].append(epoch_precision_max)
        df_dict['max_precision_epoch'].append(epoch_precision_max_epoch)
        df_dict['max_recall'].append(epoch_recall_max)
        df_dict['max_recall_epoch'].append(epoch_recall_max_epoch)

    # Create dataframe
    df = pd.DataFrame.from_dict(df_dict)
    df.set_index('run_name', inplace=True)
    df.sort_values('max_binary_accuracy', ascending=False, inplace=True)
    return df

In [162]:
present_tensorboard_logs()

,max_binary_accuracy,max_binary_accuracy_epoch,max_precision,max_precision_epoch,max_recall,max_recall_epoch
run_name,,,,,,
run_pairwisecorr__boruta__umap_01-14-2024-11:40:33,0.714376,4,0.370923,11,0.370079,10
run_vif__umap_01-14-2024-12:13:45,0.709742,11,0.385774,5,0.654933,0
run_all_01-14-2024-11:36:15,0.706477,2,0.353408,2,0.963872,5
run_pairwisecorr__boruta_01-14-2024-11:36:51,0.698578,13,0.392012,13,0.885132,3
run_boruta__umap_01-14-2024-12:08:14,0.690363,4,0.344732,4,0.896711,0
run_vif_01-14-2024-11:44:37,0.641601,28,0.342770,28,0.928208,0
run_vif__boruta_01-14-2024-11:53:03,0.629700,44,0.343339,44,0.858268,4
run_umap_01-14-2024-12:13:13,0.582833,3,0.232641,3,0.977767,0
run_boruta_01-14-2024-12:03:10,0.581991,4,0.329438,4,0.906438,1


In [163]:
present_tensorboard_logs('model_selection')

,max_binary_accuracy,max_binary_accuracy_epoch,max_precision,max_precision_epoch,max_recall,max_recall_epoch
run_name,,,,,,
three_layer_6_hour,0.726593,14,0.382245,12,0.636406,0
three_layer_dropout_6_hour,0.718483,24,0.390595,18,0.616489,0
two_layer_dropout_12_hour,0.715776,31,0.392938,26,0.645042,3
three_layer_dropout_12_hour,0.715776,20,0.391288,17,0.617702,0
three_layer_dropout_1_hour,0.714000,31,0.382022,27,0.620897,1
three_layer_12_hour,0.710402,8,0.391703,8,0.629750,1
three_layer_1_hour,0.710000,49,0.373396,49,0.664817,0
two_layer_dropout_6_hour,0.709847,25,0.377652,23,0.630848,0
two_layer_12_hour,0.703025,16,0.382143,16,0.659870,0


In [164]:
present_tensorboard_logs('model_tuning')

['./tensorboard/model_tuning/two_layer_dropout_1_hour/0041/execution0/validation/events.out.tfevents.1705199273.pop-os.219943.125.v2', './tensorboard/model_tuning/two_layer_dropout_1_hour/0057/execution0/validation/events.out.tfevents.1705199479.pop-os.219943.173.v2', './tensorboard/model_tuning/two_layer_dropout_1_hour/0045/execution0/validation/events.out.tfevents.1705199300.pop-os.219943.137.v2', './tensorboard/model_tuning/two_layer_dropout_1_hour/0044/execution0/validation/events.out.tfevents.1705199293.pop-os.219943.134.v2', './tensorboard/model_tuning/two_layer_dropout_1_hour/0037/execution0/validation/events.out.tfevents.1705199247.pop-os.219943.113.v2', './tensorboard/model_tuning/two_layer_dropout_1_hour/0011/execution0/validation/events.out.tfevents.1705199081.pop-os.219943.35.v2', './tensorboard/model_tuning/two_layer_dropout_1_hour/0019/execution0/validation/events.out.tfevents.1705199131.pop-os.219943.59.v2', './tensorboard/model_tuning/two_layer_dropout_1_hour/0068/execu

,max_binary_accuracy,max_binary_accuracy_epoch,max_precision,max_precision_epoch,max_recall,max_recall_epoch
run_name,,,,,,
two_layer_dropout_1_hour_0030,0.772316,0,0.227684,1,1.000000,1
two_layer_dropout_1_hour_0078,0.772316,3,0.350774,2,0.999075,6
two_layer_dropout_1_hour_0047,0.772316,0,0.352900,5,1.000000,1
two_layer_dropout_1_hour_0026,0.772316,0,0.227684,1,1.000000,1
two_layer_dropout_1_hour_0070,0.772316,1,0.370771,3,0.489135,5
...,...,...,...,...,...,...
two_layer_dropout_1_hour_0027,0.227684,0,0.227684,0,1.000000,0
two_layer_dropout_1_hour_0079,0.227684,0,0.227684,0,1.000000,0
two_layer_dropout_1_hour_0004,0.227684,0,0.227684,0,1.000000,0
